Welcome to the initial draft version of the jupyter notebook i'm writing for fulfilment of this project. This project is the famous Kaggle project related to applying ML techniques on the Titanic dataset.

Author - Aausuman Deep

Github - www.github.com/aausuman

Date - 15 November, 2021

Time Initiated - 23:15

In [11]:
# Importing the necessary libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

In [15]:
# Reading in the data
training_set = pd.read_csv("/Users/aausumandeep/Documents/Gits/Titanic/titanic_data/train.csv")
testing_set = pd.read_csv("/Users/aausumandeep/Documents/Gits/Titanic/titanic_data/test.csv")

# Creating a new column for the prediction values later
testing_set["Survived_Pred"] = ""

In [16]:
# Checking the defined length of the two datasets
print(len(training_set))
print(len(testing_set))

891
418


In [17]:
training_set.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [19]:
training_set.columns
training_set.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [18]:
x_train = training_set[training_set.columns.difference(["Survived"])]
x_test = testing_set[testing_set.columns.difference(["Survived_Pred"])]
y_train = training_set['Survived']
y_test = testing_set['Survived_Pred']



model = RandomForestClassifier(n_estimators=100)
model.fit(x_train, y_train)

y_test = model.predict(x_test)



ValueError: could not convert string to float: 'C85'